#### LLama-3.2

In [1]:
import evaluate
import nltk
import requests

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MANASA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import json

# Load queries
with open('./MultiHop-RAG/dataset/MultiHopRAG.json', 'r', encoding='utf-8') as f:
    queries = json.load(f)

# Load corpus
with open('./MultiHop-RAG/dataset/corpus.json', 'r', encoding='utf-8') as f:
    corpus = json.load(f)

#First query
queries[0]

{'query': 'Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?',
 'answer': 'Sam Bankman-Fried',
 'question_type': 'inference_query',
 'evidence_list': [{'title': 'The FTX trial is bigger than Sam Bankman-Fried',
   'author': 'Elizabeth Lopatto',
   'url': 'https://www.theverge.com/2023/9/28/23893269/ftx-sam-bankman-fried-trial-evidence-crypto',
   'source': 'The Verge',
   'category': 'technology',
   'published_at': '2023-09-28T12:00:00+00:00',
   'fact': 'Before his fall, Bankman-Fried made himself out to be the Good Boy of crypto — the trustworthy face of a sometimes-shady industry.'},
  {'title': 'SBF’s trial starts soon, but how did he — and FTX — get here?',
   'author': 'Jacquelyn Melinek',
   'url': 'https://techcrunch.com/2023/10/01/ftx-lawsuit-timeline/',
   'source': 'TechCrunch',
   'catego

In [3]:
total_words = 0
doc_lengths = []
for doc in corpus:
    body = doc['body']
    word_count = len(body.split())
    total_words += word_count
    doc_lengths.append((doc['title'], word_count))

# Output
doc_lengths.sort(key=lambda x: x[1], reverse=True)
print(f"Number of documents: {len(corpus)}")
print(f"Total word count: {total_words}")
print(f"Average words per document: {total_words / len(corpus):.2f}")

top_10_docs = doc_lengths[:10]
print("Top 10 documents by word length:\n")
for i, (title, word_count) in enumerate(top_10_docs, 1):
    print(f"{i}. {title} — {word_count} words")

Number of documents: 609
Total word count: 1063319
Average words per document: 1746.01
Top 10 documents by word length:

1. 78 Absolute Best October Prime Day Deals (2023) — 12387 words
2. The best video games of the year so far — 11660 words
3. NFL Fantasy Football Week 5 Lineup Decisions: Starts, Sits, Sleepers, Busts to know for every game — 10790 words
4. NFL Fantasy Football Week 6 Lineup Decisions: Starts, Sits, Sleepers, Busts to know for every game — 10490 words
5. The best movies of 2023 so far — 9926 words
6. The people who ruined the internet — 8480 words
7. The 53 best Black Friday deals we could find at Amazon, Walmart, Target and more — 7969 words
8. The Halloween Countdown: 31 days of horror to watch — 7754 words
9. ChatGPT: Everything you need to know about the AI-powered chatbot — 7482 words
10. The obsessive tormenter who made professors’ lives miserable — 7080 words


In [4]:
# Now calculate query answer lengths
answer_lengths = [len(query["answer"].split()) for query in queries]

max_answer_length = max(answer_lengths)
avg_answer_length = sum(answer_lengths) / len(answer_lengths)

print(f"\nMaximum answer length: {max_answer_length} words")
print(f"Average answer length: {avg_answer_length:.2f} words")


Maximum answer length: 8 words
Average answer length: 1.32 words


In [5]:
title_to_doc = {doc["title"]: doc["body"] for doc in corpus}

In [ ]:
#Fill this before running further cells
DEEPINFRA_API_KEY = ""
TOGETHER_API_KEY = ""

In [25]:
from openai import OpenAI

openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

stream = False 

In [8]:
def doc_length(title):
    body = title_to_doc[title]
    return len(body.split())

In [16]:
def query_llama_deepinfra(prompt):
    response = openai.chat.completions.create(
        model="meta-llama/Meta-Llama-3-70B-Instruct",  # Confirm availability on DeepInfra
        model = "
        messages=[
            {"role": "system", "content": "You are a helpful assistant for multi-hop QA. Provide specific answer without details"},
            {"role": "user", "content": prompt}
        ],
        stream=stream
    )
    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens

    print(f"Prompt: {prompt_tokens}, Completion: {completion_tokens}, Total: {total_tokens}")
    return response.choices[0].message.content

In [61]:
def query_llama_togetherai_freemodel(prompt):
    openai_together = OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url="https://api.together.xyz/v1",
    )
    response = openai_together.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",  # Confirm availability on Together.AI
        messages=[
            {"role": "system", "content": "You are a helpful assistant for multi-hop QA. Answer very concisely without providing details. Use information from the context to answer the question. If the answer requires combining facts from different documents, do so. If context is insufficient to answer the question, answer 'Insufficient Information' "},
            {"role": "user", "content": prompt}
        ],
        stream=stream
    )
    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens

    print(f"Prompt: {prompt_tokens}, Completion: {completion_tokens}, Total: {total_tokens}")
    return response.choices[0].message.content
    

In [ ]:
def query_llama_togetherai(prompt):
    openai_together = OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url="https://api.together.xyz/v1",
    )
    response = openai.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",  # Confirm availability on DeepInfra
        messages=[
            {"role": "system", "content": "You are a helpful assistant for multi-hop QA. Provide specific answer without details"},
            {"role": "user", "content": prompt}
        ],
        stream=stream
    )
    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens

    print(f"Prompt: {prompt_tokens}, Completion: {completion_tokens}, Total: {total_tokens}")
    return response.choices[0].message.content
    

In [62]:
def build_prompt_full_corpus(query_obj):
    # Get the list of document titles from the evidence list in query_obj
    evidence_list = query_obj.get("evidence_list", [])
    evidence_titles = [ev['title'] for ev in evidence_list]
    
    # Retrieve the document bodies for each title found in title_to_doc
    context_docs = [
        f"[Doc {i+1}] {title_to_doc[title]}"
        for i, title in enumerate(evidence_titles)
        if title in title_to_doc
    ]
    
    context = "\n\n".join(context_docs)
    
    return f"""Question: {query_obj['query']}

Context:
{context}

Answer:"""

In [11]:
#Evaluation Metrics
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

def compute_mrr(preds, refs):
    ranks = [1 if pred.strip().lower() == ref.strip().lower() else 0 for pred, ref in zip(preds, refs)]
    return sum(1 / (i + 1) for i, r in enumerate(ranks) if r) / max(1, sum(ranks))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MANASA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\MANASA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MANASA\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [23]:
len(queries)

2556

In [124]:
results = []
prompts = []
greater_than_8k=[]

In [175]:
query_to_evidence_list = {}
for query in queries:
    query_to_evidence_list[query['query']] = query['evidence_list'] 

In [ ]:
for i in range(0,2557): 
    print("Query: ", i)
    query_obj = queries[i]
    gold = query_obj["answer"]
    prompt = build_prompt_full_corpus(query_obj)
    prompts.append(prompt)
    
    try:
        pred = query_llama_togetherai_freemodel(prompt)
        results.append({"query": query_obj["query"], "gold": gold, "pred": pred})
    except Exception as e:
        print("API Error:", e)
        greater_than_8k.append(i)
        pred = ""
        
    print({"query": query_obj["query"], "gold": gold, "pred": pred})

Query:  2300
Prompt: 145, Completion: 4, Total: 149
{'query': 'What is the name of the Border Patrol chief, as reported by The New York Times, who implemented a controversial policy leading to an increase in arrests, and was also mentioned in a Washington Post article for his efforts to strengthen border security?', 'gold': 'Insufficient information.', 'pred': 'Insufficient Information'}
Query:  2301
Prompt: 5794, Completion: 4, Total: 5798
{'query': "Was the report by Business Today | Latest Stock Market And Economy News India on October 7, 2022, regarding an intelligence success by Mossad, and the subsequent report by Globes English | Israel Business Arena on November 5, 2023, concerning Israel's defense reliance on technology, inconsistent in their reporting on Israel's security situation?", 'gold': 'no', 'pred': 'Inconsistent.'}
Query:  2302
Prompt: 4975, Completion: 4, Total: 4979
{'query': "Who became a notable figure in generative AI for impacting public awareness through ChatGP

In [126]:
import pickle

with open('intermediate_results-2300-2556.pkl', 'wb') as f:
    pickle.dump({
        "results": results,
        "prompts": prompts
    }, f)


In [ ]:
print("Length of results:",len(results))
print("Length of 'greater_than_8k' queries", len(greater_than_8k))

Length of results: 1945
Length of 'greater_than_8k' queries 94


In [ ]:
#greater_than_8k

[512, 525, 556, 568, 575, 584, 620, 929, 957, 968]

In [ ]:
g= greater_than_8k
for i in range(len(g)): 
    print("Query: ", g[i])
    if i in greater:
        continue
    query_obj = queries[g[i]]
    gold = query_obj["answer"]
    prompt = build_prompt_full_corpus(query_obj)
    prompts.append(prompt)
    
    try:
        pred = query_llama_togetherai_freemodel(prompt)
        results.append({"query": query_obj["query"], "gold": gold, "pred": pred})
    except Exception as e:
        print("API Error:", e)
        greater_than_8k.append(i)
        pred = ""
        
    print({"query": query_obj["query"], "gold": gold, "pred": pred})

Query:  1145
Prompt: 4175, Completion: 2, Total: 4177
{'query': 'What company, according to TechCrunch, claims to have developed a generative AI model with superior architecture while also spending billions to secure its position as the default search engine and is accused of harming news publishers’ revenues through anticompetitive practices?', 'gold': 'Google', 'pred': 'Google'}
Query:  1175
Prompt: 4154, Completion: 3, Total: 4157
{'query': "Does the 'Fortune' article suggest that Denise George was dismissed for her legal actions related to Jeffrey Epstein's estate, while the 'Sport Grill' article discusses whether Reece James should have been dismissed for an elbowing incident during a football match?", 'gold': 'Yes', 'pred': 'Yes.'}
Query:  1207
API Error: Error code: 422 - {'id': 'nntW2ry-3NKUce-927b6e1f78602b6e', 'error': {'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 8193. Given: 6629 `inputs` tokens and 2048 `max_new_tokens`', 'type': 'inval

In [ ]:
with open("results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [ ]:
with open('greater_than_8k.txt', 'w') as f:
    for num in greater_than_8k:
        f.write(f"{num}\n")

In [129]:
with open('greater_than_8k_comma_2300-2556-162.txt', 'w') as f:
    f.write(','.join(map(str, greater_than_8k)))

In [ ]:
#results_strings = ["results100.json","results500-346.json","results1000-341.json","results1320-233.json","results1573-172.json","results1749-123.json","results2000-297.json","results2300-209.json","results2556-162.json"]

In [ ]:
# results_new = []
# for filename in results_strings:
#     with open(filename, "r", encoding="utf-8") as f:
#         data = json.load(f)  # each file contains a list of {"query", "gold", "pred"} objects
#         results_new.extend(data)

# # Optional: save to a single merged file
# with open("merged_results.json", "w", encoding="utf-8") as f:
#     json.dump(results_new, f, indent=2, ensure_ascii=False)


In [136]:
has_duplicates = len({ex["query"] for ex in results_new}) < len(results_new)

In [155]:
len({ex["query"] for ex in results_new})

1757

In [158]:
seen = set()
results_non_dup = []

for ex in results_new:
    query = ex["query"]
    if query not in seen:
        seen.add(query)
        results_non_dup.append(ex)

In [159]:
len(results_non_dup)

1757

In [162]:
results_new=results_non_dup

In [163]:
#Scoring

preds = [r["pred"] for r in results_new]
refs = [r["gold"] for r in results_new]

meteor_result = meteor.compute(predictions=preds, references=refs)
rouge_result = rouge.compute(predictions=preds, references=refs)
mrr_score = compute_mrr(preds, refs)

In [164]:
#Results

print("\nEvaluation Results")
print("METEOR:", round(meteor_result["meteor"], 4))
print("ROUGE:", {k: round(v, 4) for k, v in rouge_result.items()})
print("MRR:", round(mrr_score, 4))


Evaluation Results
METEOR: 0.3624
ROUGE: {'rouge1': 0.6191, 'rouge2': 0.2217, 'rougeL': 0.6194, 'rougeLsum': 0.6187}
MRR: 0.0115


In [165]:
from bert_score import score

# Extract candidate (predicted) and reference (gold) texts
candidates = [ex["pred"] for ex in results_new]
references = [ex["gold"] for ex in results_new]

# Compute BERTScore (you can specify model_type="roberta-large" or others)
P, R, F1 = score(candidates, references, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)

# Print average scores
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall:    {R.mean().item():.4f}")
print(f"BERTScore F1:        {F1.mean().item():.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/28 [00:00<?, ?it/s]

done in 132.54 seconds, 13.26 sentences/sec
BERTScore Precision: 0.6576
BERTScore Recall:    0.6879
BERTScore F1:        0.6685


In [177]:
results_new_contexts = []

for ex in results_new:
    query = ex["query"]
    gold = ex["gold"]
    pred = ex["pred"]
    
    #query_obj = {"query": query}  # reconstructing query_obj to extract evidence_list
    evidence_list = query_to_evidence_list[query]

    evidence_titles = [ev['title'] for ev in evidence_list]
    
    context_docs = [
        f"[Doc {i+1}] {title_to_doc[title]}"
        for i, title in enumerate(evidence_titles)
        if title in title_to_doc
    ]
    
    context = "\n\n".join(context_docs)
    
    results_new_contexts.append({
        "query": query,
        "gold": gold,
        "pred": pred,
        "context": context
    })


In [179]:
#Context recall
from difflib import SequenceMatcher

def fuzzy_contains(text, substring, threshold=0.8):
    return SequenceMatcher(None, text.lower(), substring.lower()).ratio() >= threshold

context_recall_count = 0

for ex in results_new_contexts:
    context = ex.get("context", "")  # the input context given to the LLM
    gold = ex["gold"]

    if gold in context or fuzzy_contains(context, gold):
        context_recall_count += 1

context_recall = context_recall_count / len(results_new)
print(f"Context Recall: {context_recall:.4f}")


Context Recall: 0.5009


[]


In [5]:
gg = [4, 8, 15, 16, 17, 19, 22, 31, 32, 33, 35, 36, 45, 46, 51, 54, 57, 59, 60, 63, 64, 66, 70, 71, 72, 73, 76, 78, 80, 81, 84, 85, 87, 88, 90, 93, 98, 102, 104, 111, 119, 121, 123, 129, 136, 141, 145, 147, 154, 164, 165, 167, 174, 179, 187, 190, 191, 195, 198, 203, 209, 210, 212, 215, 220, 222, 223, 237, 239, 246, 249, 250, 251, 256, 262, 264, 265, 269, 272, 274, 276, 277, 278, 279, 282, 286, 289, 290, 296, 301, 303, 305, 307, 309, 310, 312, 315, 321, 322, 325, 327, 329, 331, 332, 336, 341, 346, 348, 351, 352, 353, 360, 366, 369, 370, 371, 376, 381, 382, 385, 386, 389, 390, 398, 400, 402, 404, 405, 416, 418, 421, 425, 429, 432, 435, 438, 440, 448, 451, 453, 455, 463, 464, 466, 468, 469, 472, 478, 480, 481, 487, 492, 499, 506, 510, 515, 517, 519, 520, 521, 530, 534, 537, 539, 540, 541, 542, 544, 548, 551, 553, 554, 560, 579, 580, 581, 582, 585, 589, 592, 594, 595, 598, 599, 603, 605, 606, 608, 609, 614, 615, 616, 617, 622, 626, 628, 632, 633, 640, 641, 643, 646, 648, 650, 651, 658, 663, 665, 670, 671, 672, 682, 691, 692, 694, 697, 698, 700, 702, 704, 706, 708, 710, 711, 712, 716, 718, 719, 722, 723, 724, 725, 728, 738, 739, 742, 749, 753, 756, 760, 761, 762, 765, 768, 771, 773, 776, 779, 781, 782, 784, 790, 799, 801, 804, 809, 817, 819, 821, 832, 833, 834, 840, 845, 848, 850, 852, 853, 861, 863, 867, 868, 870, 872, 874, 876, 879, 883, 884, 889, 893, 896, 897, 906, 908, 911, 912, 915, 916, 917, 921, 922, 929, 930, 932, 933, 937, 941, 942, 944, 947, 948, 951, 953, 964, 966, 972, 975, 978, 980, 991, 994, 995, 998, 1000, 1006, 1008, 1011, 1013, 1015, 1016, 1020, 1023, 1024, 1027, 1030, 1035, 1036, 1042, 1047, 1048, 1055, 1056, 1059, 1060, 1063, 1064, 1069, 1073, 1084, 1086, 1088, 1090, 1091, 1095, 1101, 1102, 1105, 1108, 1110, 1113, 1115, 1120, 1121, 1123, 1124, 1139, 1143, 1150, 1154, 1166, 1169, 1174, 1177, 1178, 1185, 1187, 1191, 1192, 1193, 1194, 1198, 1204, 1207, 1214, 1223, 1228, 1232, 1239, 1241, 1242, 1248, 1259, 1262, 1263, 1264, 1270, 1271, 1278, 1280, 1281, 1282, 1284, 1287, 1288, 1289, 1290, 1295, 1298, 1299, 1305, 1313, 1317, 1319, 1323, 1326, 1327, 1330, 1331, 1337, 1340, 1342, 1343, 1347, 1350, 1351, 1353, 1355, 1362, 1363, 1364, 1368, 1370, 1371, 1379, 1380, 1381, 1383, 1387, 1389, 1394, 1395, 1398, 1400, 1405, 1411, 1414, 1416, 1417, 1418, 1419, 1420, 1424, 1425, 1428, 1429, 1433, 1434, 1440, 1442, 1443, 1451, 1456, 1460, 1468, 1471, 1473, 1475, 1476, 1481, 1482, 1498, 1499, 1501, 1509, 1515, 1521, 1528, 1535, 1537, 1538, 1539, 1542, 1544, 1545, 1547, 1549, 1551, 1553, 1554, 1555, 1561, 1564, 1565, 1567, 1574, 1577, 1580, 1582, 1583, 1584, 1585, 1588, 1599, 1600, 1602, 1603, 1607, 1610, 1611, 1615, 1616, 1617, 1625, 1630, 1634, 1638, 1641, 1642, 1643, 1644, 1648, 1652, 1653, 1654, 1659, 1663, 1665, 1671, 1674, 1677, 1682, 1687, 1693, 1697, 1701, 1702, 1705, 1709, 1713, 1720, 1723, 1725, 1728, 1731, 1742, 1743, 1746, 1756, 1761, 1763, 1767, 1770, 1777, 1784, 1787, 1790, 1791, 1792, 1794, 1798, 1799, 1802, 1803, 1813, 1814, 1823, 1824, 1828, 1829, 1831, 1834, 1837, 1839, 1840, 1841, 1854, 1856, 1872, 1873, 1874, 1875, 1876, 1877, 1881, 1884, 1886, 1888, 1890, 1892, 1905, 1912, 1913, 1915, 1916, 1918, 1926, 1930, 1932, 1935, 1939, 1941, 1942, 1947, 1948, 1953, 1955, 1956, 1961, 1966, 1967, 1969, 1972, 1973, 1977, 1978, 1979, 1980, 1984, 1985, 1992, 1993, 1994, 1998, 1999, 2000, 2006, 2011, 2016, 2018, 2019, 2025, 2027, 2030, 2035, 2037, 2041, 2042, 2043, 2045, 2048, 2055, 2058, 2059, 2071, 2073, 2075, 2077, 2080, 2081, 2088, 2091, 2098, 2103, 2106, 2111, 2115, 2116, 2118, 2125, 2130, 2135, 2137, 2138, 2139, 2147, 2152, 2157, 2160, 2167, 2174, 2175, 2177, 2189, 2190, 2192, 2193, 2194, 2196, 2201, 2206, 2207, 2214, 2215, 2216, 2218, 2224, 2225, 2228, 2230, 2236, 2238, 2239, 2240, 2245, 2249, 2250, 2254, 2262, 2264, 2266, 2273, 2276, 2277, 2278, 2280, 2281, 2285, 2286, 2287, 2289, 2292, 2293, 2295, 2298, 2299, 2303, 2307, 2309, 2311, 2312, 2315, 2316, 2317, 2318, 2320, 2321, 2324, 2327, 2328, 2331, 2334, 2335, 2338, 2344, 2346, 2347, 2350, 2351, 2352, 2355, 2356, 2359, 2365, 2373, 2377, 2378, 2379, 2383, 2387, 2389, 2390, 2391, 2393, 2394, 2395, 2399, 2404, 2405, 2407, 2408, 2412, 2413, 2415, 2417, 2421, 2422, 2427, 2430, 2432, 2433, 2434, 2435, 2437, 2440, 2443, 2446, 2448, 2449, 2464, 2466, 2468, 2473, 2475, 2480, 2487, 2488, 2494, 2495, 2496, 2499, 2501, 2504, 2505, 2512, 2513, 2515, 2520, 2522, 2523, 2528, 2530, 2537, 2538, 2540, 2541, 2543, 2548, 2552, 2553]

In [6]:
from collections import Counter
duplicates = [num for num, count in Counter(gg).items() if count > 1]
print(duplicates)
len(gg)

[]


800

In [13]:
results_rand = []
prompts_rand = []

query_obj = query_obj_rand
gold = query_obj["answer"]
prompt = build_prompt_full_corpus(query_obj)
prompts_rand.append(prompt)

try:
    pred = query_llama_deepinfra(prompt)
except Exception as e:
    print("API Error:", e)
    pred = ""

results_rand.append({"query": query_obj["query"], "gold": gold, "pred": pred})

['Amazon’s Tye Brady discusses generative AI, humanoid robots and mobile manipulation', 'The best portable chargers for your gear', '78 Absolute Best October Prime Day Deals (2023)', "Amazon sellers sound off on the FTC's 'long-overdue' antitrust case"]
API Error: Error code: 400 - {'error': {'message': 'Requested input length 22965 exceeds maximum input length 8191'}}
